In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import time

import json

from skrub import TableReport
from ollama import Client
from pdfquery import PDFQuery
from pprint import pprint
from IPython.display import display, HTML

import bid_utils

In [2]:
## load the dfs built in 1_collect_files
rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data_input =  r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_input"
rep_data_output = r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output"

path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")
df_EBP = pd.read_pickle(path_df_EBP)

path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")
df_consult = pd.read_pickle(path_df_consult)

path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")
df_rejet = pd.read_pickle(path_df_rejet)


In [3]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])

#df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].strip()
df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].str.strip()

In [4]:
# define a unique 'no EBP xx'  for each combinaison of SPS+Ville+Entreprise+Mission, for all related files
#
mask = df_consult_ebp["ID EBP"] == "no EBP"

# Create the combined series only for the masked rows
combined_series_for_update = (
    df_consult_ebp.loc[mask, 'SPS Name'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Ville'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Entreprise'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Mission'].astype(str)
)

# Generate unique IDs for this series
unique_ids, _ = pd.factorize(combined_series_for_update)

# Assign back to the original DataFrame using the mask
df_consult_ebp.loc[mask, "ID EBP"] = "no EBP " + (unique_ids + 1).astype(str)


In [5]:
# Add columns "AO_docs" (True/False), "AO_Doc_type"(CCTP, CCAP, RC, AAPC, Memo_tech), "Commande"(True/False)

mots_cles_a_exclure = ["plan ", "assurance", "honneur", "plans", "coupe", "vue", "facade", "lot"]

# Set AO_doc_type
# Initialize all with default value
df_consult_ebp['AO_docs'] = (
    (df_consult_ebp['file_path'].str.lower().str.contains('devis') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul')) | 
    (df_consult_ebp['file_path'].str.lower().str.contains(r'\\adm') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul'))
)
mask = df_consult_ebp['AO_docs'] == True

# Création d'un masque d'exclusion
exclusion_mask = df_consult_ebp.loc[mask, 'file_path'].str.lower().apply(
    lambda x: any(mot in x for mot in mots_cles_a_exclure)
)

# Sélection des lignes à inclure (qui ne contiennent pas les mots-clés à exclure)
inclusion_mask = mask.copy()
inclusion_mask.loc[mask] = ~exclusion_mask

df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCTP', case=False, na=False),'AO_doc_type'] = 'CCxP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCP', case=False, na=False), 'AO_doc_type'] = 'CCxP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('programme', case=False, na=False), 'AO_doc_type'] = 'CCxP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCAP', case=False, na=False), 'AO_doc_type'] = 'CCxP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('DCE', case=False, na=False), 'AO_doc_type'] = 'DCE'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('Lettre', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('courier', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('consult', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('AAPC', case=False, na=False), 'AO_doc_type'] = 'AAPC'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('PA-', case=False, na=False), 'AO_doc_type'] = 'Procedure Adaptee'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('RC', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('glement', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('planning', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('phasag', case=False, na=False), 'AO_doc_type'] = 'Planning'

In [6]:
list_file_AO_notype = ""
mask2 = (df_consult_ebp['AO_docs'] == True) &  (df_consult_ebp['AO_doc_type'] == 'no type')
list_file_AO_notype = [df_consult_ebp.loc[mask2, 'file_name'], df_consult_ebp.loc[mask2, 'file_path']]
for index, row in df_consult_ebp.loc[mask2, ['file_name', 'file_path']].iterrows():
    file = row['file_name']
    file_path = row['file_path'].strip()

    if any(mot in file.lower() for mot in mots_cles_a_exclure):
        pass

    bid_utils.path_to_link(file_path)


In [7]:


#df_consult_ebp[df_consult_ebp['file_name'] == "PA-SPSLORIENT.pdf"]
df_consult_ebp[df_consult_ebp['file_name'].str.strip() == "PA-SPSLORIENT.pdf"]
file_path = df_consult_ebp[df_consult_ebp['file_name'].str.strip() == "PA-SPSLORIENT.pdf"]['file_path'].iloc[0].strip()
#print(f"**{file_path}**")
bid_utils.path_to_link(file_path, option=None)


In [8]:
df_consult_ebp['AO_docs'].value_counts()

AO_docs
False    152162
True       3095
Name: count, dtype: int64

In [9]:
print(len(df_consult_ebp['AO_doc_type']))
print(df_consult_ebp['AO_doc_type'].value_counts().sum())
df_consult_ebp['AO_doc_type'].value_counts()


155257
1137


AO_doc_type
CCxP                 467
Reglement            349
DCE                  137
Lettre Consult        79
Planning              65
AAPC                  30
Procedure Adaptee     10
Name: count, dtype: int64

In [10]:
df_consult_ebp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155257 entries, 0 to 155256
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   SPS Name        155257 non-null  object
 1   Ville           155257 non-null  object
 2   Entreprise      155257 non-null  object
 3   Mission         155257 non-null  object
 4   statut_consult  155257 non-null  object
 5   file_path       155257 non-null  object
 6   ID EBP          155257 non-null  object
 7   file_name       155257 non-null  object
 8   AO_docs         155257 non-null  bool  
 9   AO_doc_type     1137 non-null    object
dtypes: bool(1), object(9)
memory usage: 10.8+ MB


In [11]:
TableReport(df_consult_ebp)

Processing column  10 / 10


,,,,,,,,,,


In [12]:
# Creation d'un DF_consult_elevated avec une ligne par "EBP ID", 
# et l'ajout des features resultants de l'extraction 

df_consult_ao = df_consult_ebp[df_consult_ebp['AO_docs']==True]
df_consult_elevated = df_consult_ao.drop_duplicates(subset=["ID EBP"], keep="first")

# Suppression des colonnes desormais inutiles
df_consult_elevated = df_consult_elevated.drop(columns=['file_path','file_name','AO_docs','AO_doc_type'])

df_consult_elevated['lieu'] = ""
df_consult_elevated['type travaux'] = ""
df_consult_elevated['duree travaux'] = 0
df_consult_elevated['prix travaux'] = 0.0
df_consult_elevated['maitre ouvrage'] = ""
df_consult_elevated['maitre oeuvre'] = ""

In [13]:
df_consult_elevated.info()


<class 'pandas.core.frame.DataFrame'>
Index: 923 entries, 0 to 155112
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SPS Name        923 non-null    object 
 1   Ville           923 non-null    object 
 2   Entreprise      923 non-null    object 
 3   Mission         923 non-null    object 
 4   statut_consult  923 non-null    object 
 5   ID EBP          923 non-null    object 
 6   lieu            923 non-null    object 
 7   type travaux    923 non-null    object 
 8   duree travaux   923 non-null    int64  
 9   prix travaux    923 non-null    float64
 10  maitre ouvrage  923 non-null    object 
 11  maitre oeuvre   923 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 93.7+ KB


# Test de modeles pour resumer et extraire l'info un CCxP 

In [25]:
#prompt_role = "tu es un assistant pour analyser les appels d'offres de coordination SPS, extraire les informations demandees en suivant les instructions et le format attendu"
prompt_role = "your are an assistant to analyse the bids for Coordination SPS, extract key informations followins the specified instructions in term of format and content"

prompt_task_resumer = "Extrait du texte les elements suivants : \n\
'Nom Chantier': scope du projet, objet du chantier, objectif du programme ;\n\
'Lieu du Chantier': ville, Commune, Departement, Rue ; \n\
'Maitre ouvrage': nom du Maitre d'ouvrage du projet ;\n\
'Maitre oeuvre': nom du maitre d'oeuvre du chantier  ;\n\
'Type de travaux': type et nature du travaux du chantier exemple: amenagement, construction ;\n \
'Planning previsionnel': dates du chantier par lot, par phase ; calendrier operationel ;\n\
'Prix des travaux (en euros)': exemple <10_000> ;\n\
'Duree Previsionnelle des Travaux (en mois)': exemple <8> ;\n\
'Categorie operation SPS':  I, II, ou III ;\n\
Réponds en respectant la structure ci-dessus \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
Ne mentionne pas **Autres informations:**  \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Texte: "

prompt_task_json = "En Francais, Formate les informations extraites et dans le texte, avec une structure JSON, en respectant strictement les clefs et types specifies tels que \n \
{ \
\"Nom Chantier\": \"<string>\", \
\"Lieu du Chantier\": \"<string>\",\
\"Maitre ouvrage\": \"<string>\", \
\"Maitre oeuvre\": \"<string>\", \
\"Type de Travaux\": \"<string>\", \
\"Planning previsionnel\": \"<string>\", \
\"Prix des travaux (en euros)\": \"<integer>\" , \
\"Durée Prévisionnelle des Travaux (en mois)\": \"<integer>\" ,\
} \n\
Donne des reponses uniquement pour ces clefs. en chiffre lorsque c'est possible. Si pas d'info, laisser vide  \n "

prompt_task_json_uniq = "En Francais, Formate les informations extraites et dans le texte, avec une structure JSON, en respectant strictement les clefs et types specifies tels que \n \
{ \
  \"Nom Chantier\": \"<string>\" scope du projet, objet du chantier, objectif du programme, \
  \"Lieu du Chantier\": \"<string>\"  ville  Commune  Departement Rue ,\
  \"Maitre ouvrage\": \"<string>\" nom du Maitre d'ouvrage du projet,\
  \"Maitre oeuvre\": \"<string>\" nom du maitre d'oeuvre du chantier ,\
  \"Type de Travaux\": \"<string>\" type et nature du travaux du chantier exemple: amenagement construction ,\
  \"Planning phase conception\": \"<string>\" date et duree Previsionnelle , \
  \"Planning phase realisation\": \"<string>\" date et duree Previsionnelle ,\
  \"Prix des travaux (en euros)\": \"<integer>\" ,\
  \"Durée Prévisionnelle des Travaux (en mois)\": \"<integer>\" ,\
  \"Categorie operation SPS\": \"<string>\" I II ou III,\
} \n\
Réponds en respectant la structure ci-dessus \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Texte: \n "
format_json_0 = {
    "Nom Chantier": "<string>",
    "Lieu du Chantier": "<string>",
    "Maitre ouvrage": "<string>",
    "Maitre oeuvre": "<string>",
    "Type de Travaux": "<string>",
    "Planning phase conception": "<string>",
    "Planning phase realisation": "<string>",
    "Prix des travaux": "<integer>",
    "Duree des travaux": "<integer>",
    "Categorie operation SPS": "<string>",
} 

format_json = {
    "Nom Chantier": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "Type de Travaux": "",
    "Planning phase conception": "",
    "Planning phase realisation": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
    "Categorie operation SPS": "",
} 
json_string = json.dumps(format_json, ensure_ascii=False, indent=2)

prompt_task_resumer2 = f"En Francais, extrairr du **Texte CCxP** les **Informations** suivantes : \n  \
**Informations**:\n\
Nom Chantier: scope du projet, objet du chantier, objectif du programme, \n\
Lieu du Chantier: ville  Commune  Departement Rue , \n\
Maitre ouvrage: nom du Maitre d'ouvrage du projet,\n\
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,\n\
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Planning phase conception: date et duree Previsionnelle, \n\
Planning phase realisation: date et duree Previsionnelle, \n\
Prix des travaux : en euros , type integer \n\
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer \n\
Categorie operation SPS:  I II ou III, \n\n\
**Instructions**: \n\
Donne uniquement les informations disponibles dans le texte CCxP, sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Réponds uniquement aux informations demandées qui sont toutes dans le texte\n\
**Texte CCxP**: \n "

prompt_task_json_uniq2 = f"En Francais, extraire du **Texte CCxP** les **Informations** suivantes : \n  \
**Informations**:\n\
Nom Chantier: scope du projet, objet du chantier, objectif du programme, \n\
Lieu du Chantier: ville  Commune  Departement Rue , \n\
Maitre ouvrage: nom du Maitre d'ouvrage du projet,\n\
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,\n\
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Planning phase conception: date et duree Previsionnelle, \n\
Planning phase realisation: date et duree Previsionnelle, \n\
Prix des travaux : en euros , type integer \n\
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer \n\
Categorie operation SPS: categorie ou type  I II ou III ou 1,2, 3, \n\n\
**Instructions**: \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n\n \
Réponds uniquement avec la structure JSON ci-dessus : \n{json_string}\n\n\
**Texte CCxP**: \n "

"""
prompt_task_json_uniq3 = f"Améliore et Formate la reponse, \
se focalisant sur les informations demandées presentent dans le **Texte CCxP** . N'invente rien. \
et en repectant exclusivement le format JSON demandé: \n{json_string}\n\n \
**Texte CCxP**: \n "
"""

prompt_task_json_uniq3 = f"Format the output, focusing on informations requested and available within **Texte CCxP** . Do not invent anything. \
Strickly follow the JSON format request. Write in French: \n{json_string}\n\n \
**Texte CCxP**: \n "

In [15]:
bid_utils.print_text_wrapped(prompt_task_json_uniq2)

En Francais, extrait du Texte CCxP les informations suivantes :
  Informations:
Nom Chantier: scope du projet, objet du chantier, objectif du programme,
Lieu du Chantier: ville  Commune  Departement Rue ,
Maitre ouvrage: nom du Maitre d'ouvrage du projet,
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,
Planning phase conception: date et duree Previsionnelle,
Planning phase realisation: date et duree Previsionnelle,
Prix des travaux : en euros , type integer
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer
Categorie operation SPS: categorie ou type  I II ou III ou 1,2, 3,

Instructions:
Donne uniquement les informations disponibles dans le texte , sans interpretation ou
estimation.
Donne des reponses chiffrées et quantifiées lorsque c'est possible.
La reponse doit etre ecrite en francais et avoir un maximum 900 mots

 Réponds uniquement avec la structure JSON ci-dessus :
{
 

In [16]:
# Init Ollama, with list of model, prompt

ollama_url = "http://localhost:11434"

# Get an ollama client
llmclient = Client(host=ollama_url)
model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0,
    "top_p": 0.9,
}
model_options_json = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0,
    "top_p": 0.9,
    "format": "json"
}

# 'mistral-small3.1=14G  llama3.2:latest=2G gemma3:4b=3.3G
#list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]



In [26]:
## Run all models for the same task (prompt + file) for comparison
##

def run_list_model_chat(text_from_file):
    for i in range(len(list_model)):
        model_name = list_model[i]
        print("\n==========================================")
        print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
        
        #Init system:
        llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])

        #Resumer
        prompt_full_r = prompt_task_resumer2 + text_from_file
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
        pprint(result, compact=True)
        bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)

        #JSON
        prompt_full_j = prompt_task_json_uniq3 + result.message.content
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
        pprint(result, compact=True)
        bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")


def run_list_model_chat_1proj_multiple_file():

    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCxP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCxP")
    
    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(1, len(matching_ebp_ids))
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")
            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run models pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files))
                run_list_model_chat(text_multi_files)


In [29]:
## Liste all files for given consultation type CCTP (EBP IP)
## Run le modele pour resumer et json

# Preferable de faire une extraction par type de fichier (CCTP, CCAP, ..) avec un prompt dédié.
# donc regroupement par type de fichier
"""
def run_1model_nCCxP_proj(model_name="gemma2:9b", nb_of_projet_to_test = 1):
    #model_name = "gemma2:9b"
    #nb_of_projet_to_test = 5
    count_id = 1

    # liste des ID correspondants au mask
    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCxP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCxP")
    print(f"on teste l'extraction avec {nb_of_projet_to_test} projets, par le modele {model_name}\n")

    #Init system:
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])

    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

        # Randomly select the desired number of unique EBP IDs
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

        #print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run model pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files))

                #Resumer
                prompt_full = prompt_task_resumer + text_multi_files
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full}])
                bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n")

                #JSON
                prompt_full = prompt_task_json + text_multi_files
                result = llmclient.chat(model=model_name, options=model_options_json, messages=[{'role':'user','content':prompt_full}])
                bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")

                cctp_json = bid_utils.print_json_info_cctp(result.message.content)
                
                # extrait les infos du JSON
                if cctp_json != "":
                    parsed_json = {} # Initialize an empty dictionary
                    try:
                        parsed_json = json.loads(cctp_json)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON string: {e}")
                    except TypeError:
                        print("Input is not a string.") 
                    
                    # Extrait chaque clef
                    lieu_value = parsed_json.get("Lieu du Chantier")
                    type_travaux_value = parsed_json.get("Type de Travaux")
                    planning_value = parsed_json.get("Planning previsionnel")
                    duree_travaux_value = parsed_json.get("Durée Prévisionnelle des Travaux (en mois)")
                    prix_travaux_value = parsed_json.get("Prix des travaux (en euros)")
                    moa_value = parsed_json.get("Maitre ouvrage")
                    moe_value = parsed_json.get("Maitre oeuvre")

                    # Mise à jour du df_consult_elevated
                    mask = df_consult_elevated['ID EBP'] == ebp_id
                    if lieu_value is not None:
                        df_consult_elevated.loc[mask, 'lieu'] = lieu_value
                    if type_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                    if duree_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'duree travaux'] = duree_travaux_value
                    if planning_value is not None:
                        df_consult_elevated.loc[mask, 'planning'] = planning_value
                    if prix_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value
                    if moa_value is not None:
                        df_consult_elevated.loc[mask, 'maitre ouvrage'] = moa_value
                    if moe_value is not None:
                        df_consult_elevated.loc[mask, 'maitre oeuvre'] = moe_value
            else:
                print("Pas de fichiers consult à processer")
            count_id += 1
    else:
        print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")

"""

def run_1model_gen_nCCxP_proj(model_name="gemma2:9b", nb_of_projet_to_test = 1):
    #model_name = "gemma2:9b"
    #nb_of_projet_to_test = 5
    count_id = 1

    # liste des ID correspondants au mask
    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCxP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCxP")
    print(f"on teste l'extraction avec {nb_of_projet_to_test} projets, par le modele {model_name}\n")

    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

        # Randomly select the desired number of unique EBP IDs
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

        #print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run model pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files))

                #Init system:
                llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
                
                #Resumer
                prompt_full_r = prompt_task_resumer2 + text_multi_files
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
                pprint(result, compact=True)
                bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)

                #JSON
                prompt_full_j = prompt_task_json_uniq3 + result.message.content
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
                pprint(result, compact=True)
                bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")

                #cctp_json = bid_utils.print_json_info_cctp(result.message.content)
                parsed_json = json.loads(result.message.content)

                # extrait les infos du JSON
                if parsed_json != "":
                    parsed_json = {} # Initialize an empty dictionary
                    try:
                        parsed_json = json.loads(result.message.content)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON string: {e}")
                    except TypeError:
                        print("Input is not a string.") 
                    
                    # Extrait chaque clef
                    lieu_value = parsed_json.get("Lieu du Chantier")
                    type_travaux_value = parsed_json.get("Type de Travaux")
                    planning_concept_value = parsed_json.get("Planning phase conception")
                    planning_real_value = parsed_json.get("Planning phase realistion")
                    duree_travaux_value = parsed_json.get("Duree des travaux")
                    prix_travaux_value = parsed_json.get("Prix des travaux")
                    cat_sps_value = parsed_json.get("Categorie operation SPS")
                    moa_value = parsed_json.get("Maitre ouvrage")
                    moe_value = parsed_json.get("Maitre oeuvre")

                    # Mise à jour du df_consult_elevated
                    mask = df_consult_elevated['ID EBP'] == ebp_id
                    if lieu_value is not None:
                        df_consult_elevated.loc[mask, 'lieu'] = lieu_value
                    if type_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                    if duree_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'duree travaux'] = duree_travaux_value
                    if planning_concept_value is not None:
                        df_consult_elevated.loc[mask, 'planning conception'] = planning_concept_value
                    if planning_real_value is not None:
                        df_consult_elevated.loc[mask, 'planning realisation '] = planning_real_value
                    if prix_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value
                    if moa_value is not None:
                        df_consult_elevated.loc[mask, 'maitre ouvrage'] = moa_value
                    if moe_value is not None:
                        df_consult_elevated.loc[mask, 'maitre oeuvre'] = moe_value
                    if cat_sps_value is not None:
                        df_consult_elevated.loc[mask, 'Categorie operation SPS'] = cat_sps_value
            else:
                print("Pas de fichiers consult à processer")
            count_id += 1
    else:
        print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")



In [27]:
## Teste les modeles 

#list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]
# gemma3:12b : very long , not performing so well.
# mistral:7b : does not respect instructions (info request, JSON structure)
# "qwen3:8b"
list_model = ["llama3.1:8b", "qwen3:8b"]

# Select 1 file for the test
condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                    (df_consult_ebp['AO_doc_type'] == "CCxP") 

matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCxP")
if len(matching_ebp_ids) > 0:
    # Determine how many unique IDs we can actually select (min of 5 or available count)
    #num_to_select = min(1, len(matching_ebp_ids))
    selected_ebp_id = np.random.choice(matching_ebp_ids, 1, replace=False)
    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == selected_ebp_id[0]].iloc[0]
    print(f"ID:{selected_ebp_id} statut_consult: {filtered_row['statut_consult']}")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
    
    text_multi_files = ""
    file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == selected_ebp_id[0]) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path'].iloc[0]
    #print("File ==> ", file_consult)
    bid_utils.path_to_link(file_consult.strip(), option=None)
    text_to_analyse = bid_utils.loadpdf_as_text(file_consult.strip())

# Compare model results for same file same prompt , models to test is defined in list_model
#run_list_model_chat(text_to_analyse)   #best model so far: llama3.1:8b

#or Compare model results for same projet (with multiple files), same prompt , models to test is defined in list_model
run_list_model_chat_1proj_multiple_file()



il y a 292 projets avec au moins 1 document du type CCxP
ID:['no EBP 14102'] statut_consult: Devis
Ville:NANTES  Entreprise:NM  Mission:AMENAGEMENT DES ESPACES PUBLICS


il y a 292 projets avec au moins 1 document du type CCxP
ID:no EBP 3601 statut_consult: Perdu
Il y a 2 fichiers dans la dossier AO consultation pour ce projet no EBP 3601
Ville:MORTAGNE SUR SEVRE  Entreprise:RES ST ALEXANDRE  Mission:EHPAD Residence St Alexandre


Run models pour les fichier(s) selectionné(s), lg text input: 120401

===== Test model :1/2: llama3.1:8b =========

ChatResponse(model='llama3.1:8b', created_at='2025-06-10T19:29:21.0685477Z', done=True, done_reason='stop', total_duration=165263642600, load_duration=15589800, prompt_eval_count=4096, prompt_eval_duration=106861419400, eval_count=569, eval_duration=58383964400, message=Message(role='assistant', content="Il semble que vous ayez fourni un document complet relatif à un appel d'offres pour la mise aux normes de sécurité et les travaux de désenfumage sur l'EHPAD Saint Alexandre à Mortagne-sur-Sevre. Voici une synthèse des informations clés :\n\n**Contexte**\n\n- L'opération consiste en une mise en conformité de la sécurité incendie (comprenant les travaux de désenfumage) pour répondre aux injonctions de la Commission de Sécurité et à l'ensemble des observations du rapport de SOCOTEC.\n- L'EHPAD Saint Alexandre est un établissement médico-social public hébergeant des personnes

In [ ]:
#run_1model_nCCxP_proj(model_name="llama3.1:8b", nb_of_projet_to_test = 2)
run_1model_gen_nCCxP_proj(model_name="llama3.1:8b", nb_of_projet_to_test = 2)

il y a 292 projets avec au moins 1 document du type CCxP
on teste l'extraction avec 2 projets, par le modele llama3.1:8b


1/2 => ID:no EBP 14174 statut_consult: Devis
Il y a 1 fichiers dans la dossier AO consultation pour ce projet no EBP 14174
Ville:TOURS  Entreprise:TOURS AGGLO  Mission:Extension complexe sportif Hallebardier


Run model pour les fichier(s) selectionné(s), lg text input: 35911
ChatResponse(model='llama3.1:8b', created_at='2025-06-10T19:46:06.272632Z', done=True, done_reason='stop', total_duration=158486087500, load_duration=14322700, prompt_eval_count=4096, prompt_eval_duration=107440423500, eval_count=487, eval_duration=51029747600, message=Message(role='assistant', content="Il semble que vous ayez affaire à un contrat de prestations intellectuelles, spécifiquement pour la mise en place d'un système de gestion de projet. Voici les principales clauses qui ressortent :\n\n1. **Garantie des prestations** : Les prestations sont garanties pendant 1 an à compter de la date de notification de la décision d'admission.\n2. **Pénalités** :\n   - **Pénalité de retard** : Le titulaire du marché encourt une pénalité fixée à 1/2000 par jour de retard, sans mise en demeure préalable.\n   - **Pénalité pour travail dissimulé** : Si le titulaire ne s'acquitte pas des formalités prévues par le Code du travail 

In [ ]:
print(len(df_consult_elevated[df_consult_elevated['type travaux'] != '']))
df_consult_elevated[df_consult_elevated['type travaux'] != ''].head(20)